# Import lib

In [2]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q
!pip install accelerate -U
!pip install arabert
!pip install datasets
!nvidia-smi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 94.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 46.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 15.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.5/268.5 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 122.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 88.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 104.3 MB/s eta 0:00

In [3]:
import re
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
import string
from transformers import pipeline, set_seed
from datasets import load_dataset
import pandas as pd
from datasets import load_dataset, load_metric, Dataset
from tqdm import tqdm
import torch
from sklearn.utils import shuffle

from transformers import DataCollatorForSeq2Seq
from transformers import TrainingArguments, Trainer

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [5]:
def is_any_word_in_text(substrings, text):
    pattern = '|'.join([re.escape(substring) for substring in substrings])
    match = re.search(pattern, text)
    return bool(match)

In [6]:
def filterDataframeByKeywords(df, column_name, search_keywords):
    df['Match'] = df[column_name].apply(lambda x: is_any_word_in_text(search_keywords, x))
    filtered_df = df[df['Match']]
    filtered_df = filtered_df.drop(["Match"], axis=1).reset_index(drop=True)
    return filtered_df

In [7]:
def combineDataframes(dataframes):
    combined_df = pd.concat(dataframes, ignore_index=True)
    combined_df.drop_duplicates(inplace=True)
    return combined_df

In [8]:
def extractHistoricalData(df, column_name):
    pattern = r'\b\d{4}\b'

    filtered_df = df[df[column_name].str.contains(pattern, regex=True)]
    return filtered_df

In [9]:
def count_record_with_words(df, col_name, keywords_list):
    count = 0

    for index, row in df.iterrows():
        text = row[col_name]
        if any(keyword in text for keyword in search_keywords):
            count += 1

    print(f"Number of records containing any of the search keywords: {count}")

In [10]:
keywords_list = ["الله"
                 ,"محمد علي"
                 , "الملك"
                 , "مدينة"
                 , "رومان"
                 , "إمبراطور"
                 , "شعب"
                 , "بريطانيا"
                 , "عبد الناصر"
                 ,"القاهرة"
                 , "أكتوبر"
                 ,"فاروق"
                 , "الحياة"
                 , "الإسكندرية"
                 , "القديم"
                 , "عصر"
                 , "مسلم"
                 ,  "إسلام"
                 , "العالم"
                 , "عمر"
                 , "الفرنسية"
                 , "عرابي"
                 ,  'مصر'
                 , 'تاريخ'
                 , 'حضارة'
                 , 'سيسي'
                 , 'باشا'
                 , 'ثورة'
                 , 'عربي'
                 , 'مصري'
                 , 'مماليك'
                 , 'دولة'
                 , 'دين'
                 , 'حرب'
                 , 'جيش'
                 , 'حكم'
                 , 'بلاد'
                 , "رئيس"
                , "العراق"
                , "فتح"
                , "السلطان"
                , "بابل"
                , "قوات"
                , "رومان"
                , "فرنسا"]

# Working on xlsum dataset

In [11]:
dataset_xlsum = load_dataset("csebuetnlp/xlsum", "arabic")

split_lengths = [len(dataset_xlsum[split])for split in dataset_xlsum]

print(f"Split lengths: {split_lengths}")
print(f"Features: {dataset_xlsum['train'].column_names}")
print("\nText:")

print(dataset_xlsum["test"][1]["text"])

print("\nSummary:")

print(dataset_xlsum["test"][1]["summary"])

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset xlsum downloaded and prepared to /root/.cache/huggingface/datasets/csebuetnlp___xlsum/arabic/2.0.0/518ab0af76048660bcc2240ca6e8692a977c80e384ffb18fdddebaca6daebdce. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Split lengths: [37519, 4689, 4689]
Features: ['id', 'url', 'title', 'summary', 'text']

Text:
ألمانيا تواجه مصاعب في التعامل مع الأعداد المتزايدة من طالبي اللجوء وقال وزير الاقتصاد سيغمار غابريل إن مواطني هذه الدول على الأرجح لن يمنحوا حق اللجوء إلى ألمانيا. وتأتي هذه الخطوة كجزء من إجراءات جديدة تهدف إلى تشديد القيود في بلد استقبل العام الماضي أكثر من 1.1 مليون شخص من طالبي اللجوء. وجاءت تصريحات غابريل بعد أن عقد الحزب الديمقراطي الاشتراكي محادثات مع الحزب المسيحي الديمقراطي الذي تنتمي إليه المستشارة الألمانية انغيلا ميركل وشريكهما الأصغر البافاري في الائتلاف الحكومي وهو الاتحاد المسيحي الاجتماعي. وقال غابريل "المزاج العام (في المحادثات) جيد"، حسبما نقلت عنه وكالة رويترز. مواضيع قد تهمك نهاية وردت المغرب بالفعل على هذا المقترح بالقول إنها ستعيد أيا من مواطنيها الذين وصلوا إلى ألمانيا بصورة غير قانونية. واتفق شركاء الائتلاف الحاكم أيضا على أن المهاجرين الذين يحصلون على حق لجوء مقيد بشروط محددة لن يكون بإمكانهم جلب ذويهم إلى ألمانيا لمدة عامين. كما اتفق الائتلاف على تسريع ترحيل طالبي ال

In [12]:
dataset_xlsum

DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'title', 'summary', 'text'],
        num_rows: 37519
    })
    test: Dataset({
        features: ['id', 'url', 'title', 'summary', 'text'],
        num_rows: 4689
    })
    validation: Dataset({
        features: ['id', 'url', 'title', 'summary', 'text'],
        num_rows: 4689
    })
})

In [13]:
from datasets import concatenate_datasets


merged_xlsum_dataset = concatenate_datasets([dataset_xlsum["train"], dataset_xlsum["validation"], dataset_xlsum["test"]])

df_xlsum_merged = merged_xlsum_dataset.to_pandas()

print(df_xlsum_merged.shape)
print(df_xlsum_merged.columns)

(46897, 5)
Index(['id', 'url', 'title', 'summary', 'text'], dtype='object')


In [14]:
df_xlsum_merged.head()

,id,url,title,summary,text
0,140323_russian_troops_crimea_naval_base,https://www.bbc.com/arabic/worldnews/2014/03/1...,القوات الأوكرانية تبدأ الانسحاب من القرم,بدأت القوات الأوكرانية الانسحاب من شبه جزيرة ا...,وكان الرئيس الأوكراني المؤقت، الكسندر تورتشينو...
1,130528_egypt_nile_dam,https://www.bbc.com/arabic/middleeast/2013/05/...,هل يفرض سد النهضة الإثيوبي واقعا جديدا على مصر؟,"""هل سيتم تغيير العبارة الشهيرة للمؤرخ اليوناني...",بحلول عام 2050 ستحتاج مصر إلى 21 مليار متر مكع...
2,world-47242349,https://www.bbc.com/arabic/world-47242349,تعرف على منطقة كشمير التي تسببت بحربين بين اله...,قالت الشرطة في القطاع الهندي من إقليم كشمير إن...,وذكرت وكالة الأنباء المحلية (جي.إن.إس) أن جماع...
3,vert-cul-55078328,https://www.bbc.com/arabic/vert-cul-55078328,ماذا تعرف عن العالم الخفي للمعابد اليابانية ال...,في عام 816، تجول راهب يدعى كوكاي، في المنحدرات...,ووقع اختياره على واد عمقه 800 متر محاط بثماني ...
4,141023_yemen_hodeida,https://www.bbc.com/arabic/middleeast/2014/10/...,"اشتباك بين الحوثيين و""الحراك التهامي"" في الحدي...","أكد مصدر في ""الحراك التهامي"" لأبناء محافظة الح...",مسلح حوثي في إب وقال المصدر إن المسلحين الحوثي...


In [15]:
df_xlsum_merged = df_xlsum_merged.drop(["title", "url", "id"], axis=1)
df_xlsum_filtered = filterDataframeByKeywords(df_xlsum_merged, "summary", keywords_list)
df_xlsum_filtered['summary'], df_xlsum_filtered["text"] = df_xlsum_filtered["text"], df_xlsum_filtered['summary']
df_xlsum_filtered.rename(columns={'summary': 'text', "text": "summary"}, inplace=True)
df_xlsum_merged.head()

,summary,text,Match
0,بدأت القوات الأوكرانية الانسحاب من شبه جزيرة ا...,وكان الرئيس الأوكراني المؤقت، الكسندر تورتشينو...,True
1,"""هل سيتم تغيير العبارة الشهيرة للمؤرخ اليوناني...",بحلول عام 2050 ستحتاج مصر إلى 21 مليار متر مكع...,True
2,قالت الشرطة في القطاع الهندي من إقليم كشمير إن...,وذكرت وكالة الأنباء المحلية (جي.إن.إس) أن جماع...,True
3,في عام 816، تجول راهب يدعى كوكاي، في المنحدرات...,ووقع اختياره على واد عمقه 800 متر محاط بثماني ...,False
4,"أكد مصدر في ""الحراك التهامي"" لأبناء محافظة الح...",مسلح حوثي في إب وقال المصدر إن المسلحين الحوثي...,True


In [16]:
print(df_xlsum_filtered.shape)
for i in range(5):
    print(f"summary {i}: ", df_xlsum_filtered.loc[i, "summary"])

(28999, 2)
summary 0:  بدأت القوات الأوكرانية الانسحاب من شبه جزيرة القرم.
summary 1:  "هل سيتم تغيير العبارة الشهيرة للمؤرخ اليوناني هيرودوت أن "مصر هبة النيل"، تساؤل طرحه مدونون وناشطون مصريون على مواقع التواصل الإجتماعي بعد أن اعلنت الحكومة الإثيوبية بدء تحويل مجرى النيل الازرق أحد روافد نهر النيل تمهيدا لبناء سد "النهضة" الإثيوبي.
summary 2:  قالت الشرطة في القطاع الهندي من إقليم كشمير إن انفجار قنبلة أدى إلى مقتل 40 عنصرا على الأقل من قوات الأمن الخميس، بعد يوم من انفجار أدى لإصابة 12 تلميذا على الأقل.
summary 3:  أكد مصدر في "الحراك التهامي" لأبناء محافظة الحديدة لبي بي سي مقتل مسلح حوثي وإصابة ستة آخرين بالإضافة لجرح خمسة من مسلحي الحراك في اشتباكات اندلعت مساء امس الخميس بين الطرفين قرب ساحة اعتصام تابعة للحراك التهامي وامتدت الى عدة شوارع في مدينة الحديدة الواقعة على البحر الأحمر غربي اليمن والتي اجتاحها الحوثيون الاسبوع الماضي.
summary 4:  تمر "فولكس فاغن" بأزمة عميقة، وهي شركة كبرى يبلغ عمرها 78 عاماً. في سبتمبر/أيلول، تكشّف للعالم تلاعب الشركة في اختبارات انبعاثات الغاز. فق

# Working on labeld_validation dataset

In [17]:
val_dataset = pd.read_json('/content/labeled_validation_dataset.jsonl', lines=True).drop(["example_id"], axis=1)
val_dataset.head()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/pandas/util/_decorators.py:211 in wrapper                │
│                                                                                                  │
│   208 │   │   │   │   │   raise TypeError(msg)                                                   │
│   209 │   │   │   │   else:                                                                      │
│   210 │   │   │   │   │   kwargs[new_arg_name] = new_arg_value                                   │
│ ❱ 211 │   │   │   return func(*args, **kwargs)                                                   │
│   212 │   │                                                                                      │
│   213 │   │   return cast(F, wrapper)                                                            │
│   214                                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/pandas/util/_decorators.py:331 in wrapper                │
│                                                                                                  │
│   328 │   │   │   │   │   FutureWarning,                                                         │
│   329 │   │   │   │   │   stacklevel=find_stack_level(),                                         │
│   330 │   │   │   │   )                                                                          │
│ ❱ 331 │   │   │   return func(*args, **kwargs)                                                   │
│   332 │   │                                                                                      │
│   333 │   │   # error: "Callable[[VarArg(Any), KwArg(Any)], Any]" has no                         │
│   334 │   │   # attribute "__signature__"                                                        │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/pandas/io/json/_json.py:757 in read_json                 │
│                                                                                                  │
│    754 │   │   return json_reader                                                                │
│    755 │                                                                                         │
│    756 │   with json_reader:                                                                     │
│ ❱  757 │   │   return json_reader.read()                                                         │
│    758                                                                                           │
│    759                                                                                           │
│    760 class JsonReader(abc.Iterator, Generic[FrameSeriesStrT]):                                 │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/pandas/io/json/_json.py:913 in read                      │
│                                                                                                  │
│    910 │   │   │   else:                                                                         │
│    911 │   │   │   │   data = ensure_str(self.data)                                              │
│    912 │   │   │   │   data_lines = data.split("\n")                                             │
│ ❱  913 │   │   │   │   obj = self._get_object_parser(self._combine_lines(data_lines))            │
│    914 │   │   else:                                       

In [ ]:
val_dataset.rename(columns={'paragraph': 'text'}, inplace=True)
val_dataset

# Concatenate the xlsum dataset and the validation dataset

In [ ]:
# Concatenate the data frames vertically (along rows)
concatenated_df = pd.concat([df_xlsum_filtered, val_dataset])

# Reset the index of the concatenated data frame
concatenated_df = concatenated_df.reset_index(drop=True)
concatenated_df.tail()

In [ ]:
concatenated_df.shape

In [ ]:
for i in range(5):
    print(f"summary {i}: ", concatenated_df.loc[i, "summary"])

In [ ]:
# concatenated_df.to_csv('xlsum_val_concatenated.csv', index=False)

# preprocess the data

In [ ]:
import nltk
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import ISRIStemmer
from pyarabic.araby import strip_tashkeel

nltk.download('stopwords')
nltk.download('punkt')

# Initialize Arabic stopword list
stopwords_list = stopwords.words('arabic')

global count
count = 0

# Initialize Arabic stemmer
stemmer = ISRIStemmer()

def preprocess_arabic_text(text):
    # Text Cleaning
    text = re.sub(r'<.*?>', '', text)  # Remove HTML tags

    # Tokenization
    tokens = word_tokenize(text)

    # Stopword Removal
    tokens = [token for token in tokens if token not in stopwords_list]

    # Stemming
    tokens = [stemmer.stem(token) for token in tokens]

    # Remove diacritics
    tokens = [strip_tashkeel(token) for token in tokens]

    # Join the tokens back into a preprocessed text
    preprocessed_text = ' '.join(tokens)
    global count
    count += 1
    print(count)
    return preprocessed_text

def preprocess_arabic_summary(summary):
    # Text Cleaning
    summary = re.sub(r'<.*?>', '', summary)  # Remove HTML tags

    # Tokenization
    tokens = word_tokenize(summary)

    # Stopword Removal
    tokens = [token for token in tokens if token not in stopwords_list]

    # Remove diacritics
    tokens = [strip_tashkeel(token) for token in tokens]

    # Join the tokens back into a preprocessed summary
    preprocessed_summary = ' '.join(tokens)
    global count
    count += 1
    print(count)
    return preprocessed_summary


In [ ]:
# Preprocess the 'text' column
concatenated_df['text'] = concatenated_df['text'].apply(preprocess_arabic_text)
count = 0
# Preprocess the 'summary' column
concatenated_df['summary'] = concatenated_df['summary'].apply(preprocess_arabic_summary)

In [ ]:
for i in range(5):
    print(f"summary {i}: ", concatenated_df.loc[i, "summary"])

In [ ]:
def empty_rows(df):
    empty_index = [index for index,text in enumerate(df.text.values) if str(text).strip() == '']
    print(f"Number of empty text rows: {len(empty_index)}")
    return df.iloc[empty_index].tail()
empty_rows(concatenated_df)

In [ ]:
def delete_links(input_text):
    pettern  = r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))'''
    out_text = re.sub(pettern, ' ', input_text)
    return out_text

def delete_repeated_characters(input_text):
    pattern  = r'(.)\1{2,}'
    out_text = re.sub(pattern, r"\1\1", input_text)
    return out_text

def replace_letters(input_text):
    replace = {"أ": "ا","ة": "ه","إ": "ا","آ": "ا","": ""}
    replace = dict((re.escape(k), v) for k, v in replace.items())
    pattern = re.compile("|".join(replace.keys()))
    out_text = pattern.sub(lambda m: replace[re.escape(m.group(0))], input_text)
    return out_text

def clean_text(input_text):
    replace = r'[/(){}\[\]|@âÂ,;\?\'\"\*…؟–’،!&\+-:؛-]'
    out_text = re.sub(replace, " ", input_text)
    words = nltk.word_tokenize(out_text)
    words = [word for word in words if word.isalpha()]
    out_text = ' '.join(words)
    return out_text

def remove_vowelization(input_text):
    vowelization = re.compile(""" ّ|َ|ً|ُ|ٌ|ِ|ٍ|ْ|ـ""", re.VERBOSE)
    out_text = re.sub(vowelization, '', input_text)
    return out_text

def delete_stopwords(input_text):
    stop_words = set(nltk.corpus.stopwords.words("arabic") + nltk.corpus.stopwords.words("english"))
    tokenizer = nltk.tokenize.WhitespaceTokenizer()
    tokens = tokenizer.tokenize(input_text)
    wnl = nltk.WordNetLemmatizer()
    lemmatizedTokens =[wnl.lemmatize(t) for t in tokens]
    out_text = [w for w in lemmatizedTokens if not w in stop_words]
    out_text = ' '.join(out_text)
    return out_text

def stem_text(input_text):
    st = ISRIStemmer()
    tokenizer = nltk.tokenize.WhitespaceTokenizer()
    tokens = tokenizer.tokenize(input_text)
    out_text = [st.stem(w) for w in tokens]
    out_text = ' '.join(out_text)

    return out_text

def text_prepare(input_text, ar_text):
    out_text = delete_links(input_text)
    out_text = delete_repeated_characters(out_text)
    out_text = clean_text(out_text)
#     out_text = delete_stopwords(out_text)
    if ar_text:
        out_text = replace_letters(out_text)
        out_text = remove_vowelization(out_text)
    else:
        out_text = out_text.lower()
    return out_text


def applyingPipeline(dataframe):
    dataframe["text"] = dataframe["text"].apply(text_prepare, args=(True,))
    dataframe["summary"] = dataframe["summary"].apply(text_prepare, args=(True,))
    return dataframe


#df1 = applyingPipeline(df, "text", keywords_list)

In [ ]:
# concatenated_df = applyingPipeline(concatenated_df)

In [ ]:
# concatenated_df.to_csv('xlsum_val_concatenated_preprocessed.csv', index=False)

# read preprocessed data from csv file

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/ICMTC Competition/

/content/drive/MyDrive/ICMTC Competition


In [ ]:
concatenated_df = pd.read_csv("/content/drive/MyDrive/ICMTC Competition/dataset/xlsum_val_concatenated_preprocessed.csv")
concatenated_df.head()

,text,summary
0,وكان رئس وكر ؤقت، كسندر تورتشينوف، امر سحب قوت...,بدأت القوات الأوكرانية الانسحاب جزيرة القرم .
1,حلل عام 2050 حاج مصر 21 لير متر كعب حصت حلي حس...,`` سيتم تغيير العبارة الشهيرة للمؤرخ اليوناني ...
2,ذكر وكل باء حلة ( جي.إن.إس ) جمع `` جيش حمد ''...,قالت الشرطة القطاع الهندي إقليم كشمير انفجار ق...
3,سلح حوث اب وقل صدر سلح حوث هجم شطء حرك همي إفش...,أكد مصدر `` الحراك التهامي '' لأبناء محافظة ال...
4,صعب قرب حسب ضرر سبب ازم لـ '' لكس فغن '' . فال...,تمر `` فولكس فاغن '' بأزمة عميقة، وهي شركة كبر...


In [ ]:
def get_sliced(df, percentage = 0.8, frac = 1, random_state = 123):
    # Shuffle the DataFrame
    shuffled_df = df.sample(frac=frac, random_state=random_state)

    # Calculate the number of rows for the slice
    slice_size = int(len(shuffled_df) * percentage)

    # Slice the DataFrame
    sliced_df = shuffled_df[:slice_size]
    return sliced_df.reset_index(drop=True)

In [ ]:
sliced_df = get_sliced(concatenated_df, 0.04)

In [ ]:
sliced_df.head()

,text,summary
0,قطع حطم نقل بعن فئق ارس مهد نقل عمل بحث عسكر د...,وصلت قطعة حطام طائرة، يعتقد الخبراء أنها ربما ...
1,كامير يعل استقالته ان خار نخب خرج تحد ورب أختا...,للمرة الأولى تاريخ الاتحاد الأوروبي تصوت احدى ...
2,يزد عرض هجر نطق عرق . أضف سيد `` زلن رتد لبس ص...,قالت سيدة عراقية الطائفة الأيزيدية محتجزة تنظي...
3,اقم غيلم كسك 20 عما وكان غيلم اهم كتب شعراء لغ...,توفي الشاعر الارجنتيني الكبير خوان غيلمان عمر ...
4,ركز قضة خدم نقل قئم طبق UberPop ، ربط خدم وبين...,قضت محكمة فرنسية بتغريم شركة `` أوبر '' لخدمات...


In [ ]:
sliced_df.shape

(1166, 2)

# Pretrained Model

In [20]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from arabert.preprocess import ArabertPreprocessor

model_name="abdalrahmanshahrour/arabartsummarization"
preprocessor = ArabertPreprocessor(model_name="")

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)
pipeline = pipeline("text2text-generation",model=model,tokenizer=tokenizer)

# text = """وتحت عنوان من الكارثة إلى التحدى يبدأ الكاتب عرض الكتاب الرابع ، حيث يوضح كيف كانت إسرائيل فرحة بنصرها عام 67 وأنها ارتاحت لاعتقادها بأن هناك وقتا طويلا وطويلا جدا قبل أن يفيق العرب من صدمة 67، وكيف أن القوات الجوية للجمهورية العربية المتحدة قد فاجأتها بعد شهر واحد من نهاية حرب 67 بهجوم جوى عنيف على مواقعها فى سيناء وكان هذا إعلانا عن بداية حرب من نوع جديد هى حرب الاستنزاف التى استمرت حتى تم وقف إطلاق النار بين الطرفين فى 8 أغسطس 1970، ثم وفاة عبدالناصر وتولى أنور السادات حكم مصر واستعداده للحرب . ويتعرض الكاتب أيضا وبصورة سريعة لفلسطين والأردن وسوريا قبل أن ينتقل إلى الكتاب الخامس عن حرب أكتوبر ، حيث يعرض الخطط والاستعدادات المصرية ثم الاستعدادات الإسرائيلية ثم يبدأ بعرض وقائع الحرب بداية من الضربة الجوية وانهيار خط بارليف واختراقه ، ويتوقف الكاتب عند يوم 8 أكتوبر ، ويقول : إن هذا اليوم كان اسوأ هزيمة فى تاريخ الجيش الإسرائيلى ثم ينتقل بنا المؤلف إلى الجبهة السورية ثم يعود ثانية إلى يوميات الحرب حتى 7 9 أكتوبر إلى 9 13 أكتوبر ثم 14 أكتوبر ، ثم يعرض للثغرة أو ما عرف بعملية المزرعة الصينية يوم 16 و 15 أكتوبر والمساعدات الأمريكية الضخمة لإسرائيل ، ثم بداية الضغوط السياسية على الرئيس أنور السادات من 17 19 أكتوبر ثم ينتقل الكاتب للأحداث التى جرت من 17 20 أكتوبر وإعفاء الفريق الشاذلى من منصبه كرئيس لأركان القوات المسلحة المصرية ، وتولى الفريق الجمسى بدلا منه ثم الاتجاه إلى الموافقة على طلب وقف إطلاق النار والخلاف مع سوريا بشأن هذا الأمر ، ثم بداية الهجوم الإسرائيلى من 19 إلى 22 أكتوبر على الضفة الغربية لقناة السويس والعمليات النهائية فى سوريا 14 23 أكتوبر ، وكيف أن الملك حسين قرر دخول الحرب ضد إسرائيل يوم 9 أكتوبر ، ثم يعرض الكاتب المعركة الخاصة بالاستيلاء على مدينة السويس من 23 أكتوبر إلى 25 أكتوبر ثم تطورات هذه المعركة ، وكيف أنه مع حلول يوم السابع والعشرين من أكتوبر كان الإسرائيليون قد أسروا نحو ثمانية آلاف فرد من القوات المصرية ، أغلبهم من وحدات الإمداد والتموين"""
# text = preprocessor.preprocess(text)

# result = pipeline(text,
#             pad_token_id=tokenizer.eos_token_id,
#             num_beams=3,
#             repetition_penalty=3.0,
#             max_length=200,
#             length_penalty=1.0,
#             no_repeat_ngram_size = 3)[0]['generated_text']
# result

# Prepare data for finetuning

In [21]:
def make_dataset_from_dataframe(df, train_ratio= 0.8, val_ratio = 0.1, test_ratio = 0.1):
    # Shuffle the DataFrame
    df = shuffle(df, random_state=42)

    # Split the DataFrame into train, validation, and test sets
    train_size = int(train_ratio * len(df))
    val_size = int(val_ratio * len(df))
    test_size = len(df) - train_size - val_size


    train_df = df[:train_size]
    val_df = df[train_size : train_size + val_size]
    test_df = df[-test_size:]

    # Convert the train, validation, and test DataFrames to datasets.arrow_dataset.Dataset
    train_dataset = Dataset.from_pandas(train_df)
    val_dataset = Dataset.from_pandas(val_df)
    test_dataset = Dataset.from_pandas(test_df)

    return train_dataset, val_dataset, test_dataset

In [22]:
#train_dataset, val_dataset, test_dataset = make_dataset_from_dataframe(concatenated_df)
train_dataset, val_dataset, test_dataset = make_dataset_from_dataframe(df)

In [23]:
print(train_dataset)
print(val_dataset)
print(test_dataset)

Dataset({
    features: ['text', 'summary', '__index_level_0__'],
    num_rows: 241582
})
Dataset({
    features: ['text', 'summary', '__index_level_0__'],
    num_rows: 30197
})
Dataset({
    features: ['text', 'summary', '__index_level_0__'],
    num_rows: 30199
})


In [24]:
train_dataset = train_dataset.remove_columns('__index_level_0__')
val_dataset = val_dataset.remove_columns('__index_level_0__')
test_dataset = test_dataset.remove_columns('__index_level_0__')

In [25]:
print("\nText:")

print(train_dataset["text"][0])

print("\nSummary:")

print(train_dataset["summary"][0])


Text:
ويستكمل المنتدى مسيرة المنظمة المسكونية لحقوق الإنسان والتنمية فى الاهتمام بقضايا حقوق الإنسان، والتنمية وحوار الأديان، حيث يتوسع المنتدى فى أنشطته لتشمل التعاون مع الهيئات التابعة للأمم المتحدة وعلى رأسها المفوضية العليا لحقوق الإنسان، وإصلاح صورة المفوضية لدى الرأي العام العربي من خلال لفت نظر المفوضية السامية لضرورة الاستماع لجميع الأطراف وعدم الاعتماد فى معلوماتها على طرف واحد، وهو ما تسبب في صدور تقارير تفتقد للموضوعية والحيادية، وتسهم بشكل كبير في فقدان الثقة بعمل المفوضية وتزيد من حالة الاحتقان بينها وبين بعض الدول، خاصة دول الصراع التي تعاني من آفة الإرهاب. ويعمل المنتدى على وضع آلية جديدة لتحسين العلاقة وإعادة الثقة بين الحكومات ومنظمات المجتمع المدني خاصة الحقوقية، بعد أن أساء عدد كبير منها للعمل الحقوقي بالانحياز وعدم الحيادية حتى أصبحت كلمة حقوق الإنسان سيئة السمعة، مما أثر بشكل سلبي على عمل منظمات المجتمع المدني، ومنها التنموي والصحي والبيئي فى المنطقة العربية . وصرح أيمن نصرى رئيس المنتدى العربي الأوروبي للحوار وحقوق الإنسان تحت التأسيس، بأن الهدف من تغيير الاسم هو

# visualization

In [ ]:
text_token_len = len([tokenizer.encode(s) for s in train_dataset["text"]])

summary_token_len = len([tokenizer.encode(s) for s in train_dataset["summary"]])


fig, axes = plt.subplots(1, 2, figsize=(10, 4))
axes[0].hist(text_token_len, bins = 20, color = 'C0', edgecolor = 'C0' )
axes[0].set_title("Text Token Length")
axes[0].set_xlabel("Length")
axes[0].set_ylabel("Count")

axes[1].hist(summary_token_len, bins = 20, color = 'C0', edgecolor = 'C0' )
axes[1].set_title("Summary Token Length")
axes[1].set_xlabel("Length")
plt.tight_layout()
plt.show()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│ in <listcomp>:1                                                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2332 in encode   │
│                                                                                                  │
│   2329 │   │   │   │   the `tokenize` method) or a list of integers (tokenized string ids using  │
│   2330 │   │   │   │   method).                                                                  │
│   2331 │   │   """                                                                               │
│ ❱ 2332 │   │   encoded_inputs = self.encode_plus(                                                │
│   2333 │   │   │   text,                                                                         │
│   2334 │   │   │   text_pair=text_pair,                                                          │
│   2335 │   │   │   add_special_tokens=add_special_tokens,                                        │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2740 in          │
│ encode_plus                                                                                      │
│                                                                                                  │
│   2737 │   │   │   **kwargs,                                                                     │
│   2738 │   │   )                                                                                 │
│   2739 │   │                                                                                     │
│ ❱ 2740 │   │   return self._encode_plus(                                                         │
│   2741 │   │   │   text=text,                                                                    │
│   2742 │   │   │   text_pair=text_pair,                                                          │
│   2743 │   │   │   add_special_tokens=add_special_tokens,                                        │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_fast.py:497 in           │
│ _encode_plus                                                                                     │
│                                                                                                  │
│   494 │   │   **kwargs,                                                                          │
│   495 │   ) -> BatchEncoding:                                                                    │
│   496 │   │   batched_input = [(text, text_pair)] if text_pair else [text]                       │
│ ❱ 497 │   │   batched_output = self._batch_encode_plus(                                          │
│   498 │   │   │   batched_input,                                                                 │
│   499 │   │   │   is_split_into_words=is_split_into_words,                                       │
│   500 │   │   │   add_special_tokens=add_special_tokens,                                         │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_fast.py:425 in           │
│ _batch_encode_plus                                                                               │
│                                                                                                  │
│   422 │   │   │   pad_to_multiple_of=pad_to_multiple_of,   

# convert_examples_to_features

In [26]:
def convert_examples_to_features(example_batch):
    # max_length 1024 or 512
    input_encodings = tokenizer(example_batch['text'] , max_length = 512, truncation = True, padding="max_length", return_tensors="pt" )

    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch['summary'], max_length = 128, truncation = True )

    return {
        'input_ids' : input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }

In [27]:
train_dataset_pt = train_dataset.map(convert_examples_to_features, batched = True)
test_dataset_pt = test_dataset.map(convert_examples_to_features, batched = True)
val_dataset_pt = val_dataset.map(convert_examples_to_features, batched = True)

Map:   0%|          | 0/241582 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3619: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/30199 [00:00<?, ? examples/s]

Map:   0%|          | 0/30197 [00:00<?, ? examples/s]

In [28]:
print(train_dataset_pt)
print("\nText:")
print(train_dataset_pt["text"][0])

print("\nSummary:")
print(train_dataset_pt["summary"][0])

print("\nInput_ids:")
print(train_dataset_pt["input_ids"][0])

print("\nAttention_mask:")
print(train_dataset_pt["attention_mask"][0])

print("\nLabels:")
print(train_dataset_pt["labels"][0])

Dataset({
    features: ['text', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 241582
})

Text:
ويستكمل المنتدى مسيرة المنظمة المسكونية لحقوق الإنسان والتنمية فى الاهتمام بقضايا حقوق الإنسان، والتنمية وحوار الأديان، حيث يتوسع المنتدى فى أنشطته لتشمل التعاون مع الهيئات التابعة للأمم المتحدة وعلى رأسها المفوضية العليا لحقوق الإنسان، وإصلاح صورة المفوضية لدى الرأي العام العربي من خلال لفت نظر المفوضية السامية لضرورة الاستماع لجميع الأطراف وعدم الاعتماد فى معلوماتها على طرف واحد، وهو ما تسبب في صدور تقارير تفتقد للموضوعية والحيادية، وتسهم بشكل كبير في فقدان الثقة بعمل المفوضية وتزيد من حالة الاحتقان بينها وبين بعض الدول، خاصة دول الصراع التي تعاني من آفة الإرهاب. ويعمل المنتدى على وضع آلية جديدة لتحسين العلاقة وإعادة الثقة بين الحكومات ومنظمات المجتمع المدني خاصة الحقوقية، بعد أن أساء عدد كبير منها للعمل الحقوقي بالانحياز وعدم الحيادية حتى أصبحت كلمة حقوق الإنسان سيئة السمعة، مما أثر بشكل سلبي على عمل منظمات المجتمع المدني، ومنها التنموي والصحي والبيئي فى المنطقة العرب

# Train model

In [29]:
from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [32]:
%cd /content/drive/MyDrive/ICMTC Project/model

/content/drive/MyDrive/ICMTC Project/model


In [36]:
from transformers import TrainingArguments, Trainer
trainer_args = TrainingArguments(
    output_dir='arabert-ahram', num_train_epochs=2, warmup_steps=500,
    per_device_train_batch_size=8, per_device_eval_batch_size=8,
    weight_decay=0.1, logging_steps=10,
    evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
    gradient_accumulation_steps=8,
    load_best_model_at_end=True  # Add load_best_model_at_end
)

In [37]:
from transformers import EarlyStoppingCallback
# Define the early stopping callback
early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=3)

trainer = Trainer(model=model, args=trainer_args,
                    tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                    train_dataset=train_dataset_pt,
                    eval_dataset=val_dataset_pt,
                    callbacks=[early_stopping_callback]  # Add the early stopping callback
)



In [ ]:
trainer.train()

Step,Training Loss,Validation Loss


In [ ]:
#save model
model.save_pretrained("/content/drive/MyDrive/ICMTC Project/model/arabert-ahram")

# Save tokenizer
tokenizer.save_pretrained("/content/drive/MyDrive/ICMTC Project/model/tokenizer-ahram")

# Evaluating the model

In [ ]:
def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]

In [ ]:


def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]



def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,
                               batch_size=16, device=device,
                               column_text="article",
                               column_summary="highlights"):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):

        inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                        padding="max_length", return_tensors="pt")

        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                         attention_mask=inputs["attention_mask"].to(device),
                         length_penalty=0.8, num_beams=8, max_length=128)
        ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''

        # Finally, we decode the generated texts,
        # replace the  token, and add the decoded texts with the references to the metric.
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                clean_up_tokenization_spaces=True)
               for s in summaries]

        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]


        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    #  Finally compute and return the ROUGE scores.
    score = metric.compute()
    return score

In [ ]:
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

rouge_metric = load_metric('rouge')

In [ ]:
score = calculate_metric_on_test_ds(
    test_dataset_pt, rouge_metric, trainer.model, tokenizer, batch_size = 8, column_text = 'text', column_summary= 'summary'
)

rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )

pd.DataFrame(rouge_dict, index = ['arabert'] )

# All in one

In [ ]:
import re
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
import string
from transformers import pipeline, set_seed
from datasets import load_dataset
import pandas as pd
from datasets import load_dataset, load_metric, Dataset
from tqdm import tqdm
import torch
from sklearn.utils import shuffle

from transformers import DataCollatorForSeq2Seq
from transformers import TrainingArguments, Trainer

device = "cuda" if torch.cuda.is_available() else "cpu"

def make_dataset_from_dataframe(df, train_ratio= 0.8, val_ratio = 0.1, test_ratio = 0.1):
    # Shuffle the DataFrame
    df = shuffle(df, random_state=42)

    # Split the DataFrame into train, validation, and test sets
    train_size = int(train_ratio * len(df))
    val_size = int(val_ratio * len(df))
    test_size = len(df) - train_size - val_size


    train_df = df[:train_size]
    val_df = df[train_size : train_size + val_size]
    test_df = df[-test_size:]

    # Convert the train, validation, and test DataFrames to datasets.arrow_dataset.Dataset
    train_dataset = Dataset.from_pandas(train_df)
    val_dataset = Dataset.from_pandas(val_df)
    test_dataset = Dataset.from_pandas(test_df)

    return train_dataset, val_dataset, test_dataset

def convert_examples_to_features(example_batch):
    # max_length 1024 or 512
    input_encodings = tokenizer(example_batch['text'] , max_length = 512, truncation = True, padding="max_length", return_tensors="pt" )

    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch['summary'], max_length = 128, truncation = True )

    return {
        'input_ids' : input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }



from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/ICMTC Competition/
concatenated_df = pd.read_csv("/content/drive/MyDrive/ICMTC Competition/dataset/xlsum_val_concatenated_preprocessed.csv")

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from arabert.preprocess import ArabertPreprocessor

# "abdalrahmanshahrour/auto-arabic-summarization"
#"abdalrahmanshahrour/arabartsummarization"
model_name="ahmeddbahaa/arabert2arabert-finetuned-ar-wikilingua"
preprocessor = ArabertPreprocessor(model_name="")

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)
pipeline = pipeline("text2text-generation",model=model,tokenizer=tokenizer)
train_dataset, val_dataset, test_dataset = make_dataset_from_dataframe(concatenated_df)

train_dataset = train_dataset.remove_columns('__index_level_0__')
val_dataset = val_dataset.remove_columns('__index_level_0__')
test_dataset = test_dataset.remove_columns('__index_level_0__')

train_dataset_pt = train_dataset.map(convert_examples_to_features, batched = True)
test_dataset_pt = test_dataset.map(convert_examples_to_features, batched = True)
val_dataset_pt = val_dataset.map(convert_examples_to_features, batched = True)

from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
%cd /content/drive/MyDrive/ICMTC Competition/model


from transformers import TrainingArguments, Trainer
trainer_args = TrainingArguments(
    output_dir='arabertWiki-xlsum', num_train_epochs=4, warmup_steps=500,
    per_device_train_batch_size=2, per_device_eval_batch_size=4,
    weight_decay=0.1, logging_steps=10,
    evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
    gradient_accumulation_steps=2,
    load_best_model_at_end=True  # Add load_best_model_at_end
)

from transformers import EarlyStoppingCallback
# Define the early stopping callback
early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=5)

trainer = Trainer(model=model, args=trainer_args,
                    tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                    train_dataset=train_dataset_pt,
                    eval_dataset=val_dataset_pt,
                    callbacks=[early_stopping_callback]  # Add the early stopping callback
)

trainer.train()

#save model
model.save_pretrained("/content/drive/MyDrive/ICMTC competion/model/arabert-xlsum-model")

# Save tokenizer
tokenizer.save_pretrained("/content/drive/MyDrive/ICMTC competion/model/tokenizer")


def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]



def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]



def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,
                               batch_size=16, device=device,
                               column_text="article",
                               column_summary="highlights"):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):

        inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                        padding="max_length", return_tensors="pt")

        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                         attention_mask=inputs["attention_mask"].to(device),
                         length_penalty=0.8, num_beams=8, max_length=128)
        ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''

        # Finally, we decode the generated texts,
        # replace the  token, and add the decoded texts with the references to the metric.
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                clean_up_tokenization_spaces=True)
               for s in summaries]

        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]


        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    #  Finally compute and return the ROUGE scores.
    score = metric.compute()
    return score

rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

rouge_metric = load_metric('rouge')


score = calculate_metric_on_test_ds(
    test_dataset_pt, rouge_metric, trainer.model, tokenizer, batch_size = 8, column_text = 'text', column_summary= 'summary'
)

rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )

pd.DataFrame(rouge_dict, index = ['arabert'] )

# len corpus

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
df = pd.read_xml("/content/drive/MyDrive/ICMTC Project/LANS_DATASET/EGY_alahram_news.xml")
df.head()

,address,title,article,summary
0,http://gate.ahram.org.eg/News/2348461.aspx,جهاز مدينة السادات يشن حملة لإزالة الإعلانات ا...,وقال المهندس أمين غنيم إن الحملة أسفرت عن إزال...,صرح المهندس أمين غنيم، رئيس جهاز تنمية مدينة ا...
1,http://gate.ahram.org.eg/News/2348471.aspx,مستشار مفتي الجمهورية يشارك في أسبوع التعريف ب...,وتأتي هذه المشاركة انطلاقًا من حرص دار الإفتاء...,توجه الدكتور إبراهيم نجم مستشار مفتي الجمهورية...
2,http://gate.ahram.org.eg/News/2348457.aspx,جامعة حلوان تنظم دورة تدريب المدربين للمستوى ا...,تهدف الدورة التدريبية إلى التدريب على أحدث ما ...,تنظم وحدة برامج التنمية البشرية بالإدارة العام...
3,http://gate.ahram.org.eg/News/2348454.aspx,وزير الأوقاف يفتتح تطوير وحدة القسطرة بمستشفى ...,يأتي ذلك في إطار حرص وزارة الأوقاف على تقديم خ...,يفتتح وزير الأوقاف الدكتور محمد مختار جمعة، وح...
4,http://gate.ahram.org.eg/News/2348470.aspx,طلب إحاطة لرئيس الوزراء ووزيري الأوقاف والتنمي...,وقالت شيرين القشاش إنه منذ صدور هذا الحكم وحتى...,وجهت الدكتورة شيرين القشاش عضو مجلس النواب طلب...


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 315897 entries, 0 to 315896
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   address  315897 non-null  object
 1   title    315897 non-null  object
 2   article  315897 non-null  object
 3   summary  315897 non-null  object
dtypes: object(4)
memory usage: 9.6+ MB


In [8]:
df.describe()

,address,title,article,summary
count,315897,315897,315897,315897
unique,315654,304054,310394,309119
top,http://gate.ahram.org.eg/News/2348473.aspx,وزيرا الإسكان والشباب يبحثان توفيرعضويات بأسعا...,تم اتخاذ الإجراءات القانونية حيال كل واقعة على...,عقد الدكتور عاصم الجزار، وزير الإسكان والمرافق...
freq,244,244,360,244


In [9]:
df.drop_duplicates(inplace=True)

In [10]:
df.shape

(315654, 4)

In [11]:
df.summary.duplicated().sum()

6535

In [12]:
df.drop_duplicates(subset= "address", inplace=True)
df.drop_duplicates(subset= "title", inplace=True)
df.drop_duplicates(subset= "article", inplace=True)
df.drop_duplicates(subset= "summary", inplace=True)

In [11]:
for column in df.columns:
  df.drop_duplicates(subset= column, inplace=True)

In [16]:
df.describe()

,address,title,article,summary
count,301978,301978,301978,301978
unique,301978,301978,301978,301978
top,http://gate.ahram.org.eg/News/2348461.aspx,جهاز مدينة السادات يشن حملة لإزالة الإعلانات ا...,وقال المهندس أمين غنيم إن الحملة أسفرت عن إزال...,صرح المهندس أمين غنيم، رئيس جهاز تنمية مدينة ا...
freq,1,1,1,1


In [13]:
df.shape

(301978, 4)

In [14]:
df.article.duplicated().sum()

0

In [15]:
df.head()

,address,title,article,summary
0,http://gate.ahram.org.eg/News/2348461.aspx,جهاز مدينة السادات يشن حملة لإزالة الإعلانات ا...,وقال المهندس أمين غنيم إن الحملة أسفرت عن إزال...,صرح المهندس أمين غنيم، رئيس جهاز تنمية مدينة ا...
1,http://gate.ahram.org.eg/News/2348471.aspx,مستشار مفتي الجمهورية يشارك في أسبوع التعريف ب...,وتأتي هذه المشاركة انطلاقًا من حرص دار الإفتاء...,توجه الدكتور إبراهيم نجم مستشار مفتي الجمهورية...
2,http://gate.ahram.org.eg/News/2348457.aspx,جامعة حلوان تنظم دورة تدريب المدربين للمستوى ا...,تهدف الدورة التدريبية إلى التدريب على أحدث ما ...,تنظم وحدة برامج التنمية البشرية بالإدارة العام...
3,http://gate.ahram.org.eg/News/2348454.aspx,وزير الأوقاف يفتتح تطوير وحدة القسطرة بمستشفى ...,يأتي ذلك في إطار حرص وزارة الأوقاف على تقديم خ...,يفتتح وزير الأوقاف الدكتور محمد مختار جمعة، وح...
4,http://gate.ahram.org.eg/News/2348470.aspx,طلب إحاطة لرئيس الوزراء ووزيري الأوقاف والتنمي...,وقالت شيرين القشاش إنه منذ صدور هذا الحكم وحتى...,وجهت الدكتورة شيرين القشاش عضو مجلس النواب طلب...


In [16]:
df = df.drop(["address","title"],axis=1)
df.rename(columns={"article":"text"},inplace=True)

In [17]:
df.head()

,text,summary
0,وقال المهندس أمين غنيم إن الحملة أسفرت عن إزال...,صرح المهندس أمين غنيم، رئيس جهاز تنمية مدينة ا...
1,وتأتي هذه المشاركة انطلاقًا من حرص دار الإفتاء...,توجه الدكتور إبراهيم نجم مستشار مفتي الجمهورية...
2,تهدف الدورة التدريبية إلى التدريب على أحدث ما ...,تنظم وحدة برامج التنمية البشرية بالإدارة العام...
3,يأتي ذلك في إطار حرص وزارة الأوقاف على تقديم خ...,يفتتح وزير الأوقاف الدكتور محمد مختار جمعة، وح...
4,وقالت شيرين القشاش إنه منذ صدور هذا الحكم وحتى...,وجهت الدكتورة شيرين القشاش عضو مجلس النواب طلب...


In [18]:
df.describe()

,text,summary
count,301978,301978
unique,301978,301978
top,وقال المهندس أمين غنيم إن الحملة أسفرت عن إزال...,صرح المهندس أمين غنيم، رئيس جهاز تنمية مدينة ا...
freq,1,1


In [56]:
duplicates = df.duplicated()
df_filtered = df[~duplicates]
df_filtered = df_filtered.reset_index(drop=True)

In [57]:
df_filtered.describe()

,address,title,article,summary
count,315654,315654,315654,315654
unique,315654,304054,310394,309119
top,http://gate.ahram.org.eg/News/2348461.aspx,ضبط دراجة نارية مخالفة خلال أسبوع,تم اتخاذ الإجراءات القانونية حيال كل واقعة على...,تمكنت الأجهزة الأمنية بمختلف مديريات الأمن على...
freq,1,211,360,169


In [58]:
df_filtered.shape

(315654, 4)

In [36]:
df.article[1]

'وتأتي هذه المشاركة انطلاقًا من حرص دار الإفتاء المصرية والأمانة العامة لدور وهيئات الإفتاء في العالم على التعريف بصحيح الدين والإسلام، وتغيير الصورة النمطية المشوهة عن الإسلام والمسلمين، ومواجهة ظاهرة الإسلاموفوبيا. من جانبه صرح الدكتور إبراهيم نجم، قبيل سفره بأن فعاليات الأسبوع ستتضمن إلقاء عدد من المحاضرات حول الإسلام في عدد من الجامعات الأمريكية واللقاءات العامة، بهدف تصحيح كثير من المفاهيم الإسلامية التي تم تشويهها من قِبل الجماعات المتطرفة حيث تسببت في موجات العداء والكراهية ضد الإسلام والمسلمين في الغرب. وأضاف الدكتور نجم، أنه سيتم كذلك عقد لقاءات عامة ومحاضرات تعريفية بصحيح الإسلام، كما أن هذه اللقاءات تعد فرصة جيدة لتعزيز التواصل مع الجاليات المسلمة في الولايات المتحدة الأمريكية والإجابة عن تساؤلاتهم واستفساراتهم الدينية، فضلًا عن عقد عدد من اللقاءات الإعلامية هنا حول قضايا الإسلام والمسلمين.'

In [37]:
df.summary[1]

'توجه الدكتور إبراهيم نجم مستشار مفتي الجمهورية الأمين العام لدور وهيئات الإفتاء في العالم، إلى الولايات المتحدة الأمريكية للمشاركة في أسبوع التعريف بالإسلام في الجامعات الأمريكية.'